In [1]:
% matplotlib inline

import numpy as np
import numpy.ma as ma
import pandas as pd
from math import *
import matplotlib.pyplot as plt
import scipy.io as sio
from mpl_toolkits.basemap import Basemap, cm

import MITgcmutils as mit
from MITgcmutils import cs

from matplotlib import rcParams, rc
rcParams.update({'figure.autolayout': True})
rcParams.update({'font.family': 'serif'})
rcParams.update({'font.serif': 'Times New Roman'})



/Users/sclayton/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
## import the mean data

CRindir = '/Volumes/My Passport/coarse_run/'
HRindir = '/Volumes/My Passport/high_res/'

#import grid params for CR grid

in_file = '%sgrid/HFacC.data' % CRindir
HFCR = np.fromfile(in_file, dtype = '>f')
HFCR = HFCR.reshape(23, 160, 360)[0,:,:]

in_file = '%sgrid/XC.data' % CRindir
XC = np.fromfile(in_file, dtype = '>f')
XC = XC.reshape(160,360)

in_file = '%sgrid/YC.data' % CRindir
YC = np.fromfile(in_file, dtype = '>f')
YC = YC.reshape(160,360)

in_file = '/Users/sclayton/Documents/darwin_model/eg_grid/RA.data'
RA = np.fromfile(in_file, dtype = '>f')
RA = RA.reshape(160,360)

## pick a nutrient 0 = phosphate, 1 = nitrate, 2 = iron, 3 = silica
v=2

# import CR data

in_file = '%supflux/CR_WN.1999.data' % CRindir
CRWN = np.fromfile(in_file, dtype='>f').reshape(4,160, 360)
CRmask = ma.masked_where(HFCR == 0, CRWN[v,:,:], copy=False)

#import grid params for HR grid
HFHR = mit.rdmds('/Users/sclayton/Documents/darwin_model/e2_grid/hFacC')
XG = mit.rdmds('/Users/sclayton/Documents/darwin_model/e2_grid/XG')
YG = mit.rdmds('/Users/sclayton/Documents/darwin_model/e2_grid/YG')
RAC = mit.rdmds('/Users/sclayton/Documents/darwin_model/e2_grid/RAC')

# import HR data
in_file = '%supflux/WN.1999.data' % HRindir
HRWN = np.fromfile(in_file, dtype='>f').reshape(4, 510, 3060)
HRmask = ma.masked_where(HFHR[0,:,:] == 0, HRWN[v,:,:], copy=False)

In [ ]:
## calculate the eddy upflux

HRWNanom = np.zeros((4, 510, 3060))
CRWNanom = np.zeros((4, 160, 360))

for d in range(1, 366,1):
    infile = '%supflux/WN.1999.%03d.data' % (HRindir, d)
    tmp = np.fromfile(infile, dtype = '>f').reshape(4,510, 3060)
    HRWNanom =+ tmp - HRWN
    
    infile = '%supflux/CR_WN.1999.%03d.data' % (CRindir, d)
    tmp = np.fromfile(infile, dtype = '>f').reshape(4,160,360)
    CRWNanom =+ tmp - CRWN
    

In [ ]:
fig1 = plt.figure(1, figsize=(18,9), dpi = 600)

palette = plt.get_cmap('RdBu_r', 50)

#x=mit.rdmds('XG'); y=mit.rdmds('YG'); e=mit.rdmds('Eta',np.Inf)
mp = Basemap(projection='merc',llcrnrlat=-80,urcrnrlat=80,\
            llcrnrlon=0,urcrnrlon=360,lat_ts = 0, resolution='c')
xm, ym = mp(XC,YC)
#mp.contour(xm, ym, HFCR, [0], colors='k', lw=1)
mp.pcolormesh(xm, ym, CRWNanom[1,:,:],shading='flat', vmin= -4, vmax = 4, cmap = palette)

mp.fillcontinents(color = 'grey')
mp.drawmapboundary()
mp.drawparallels(np.arange(-80,80,20),labels=[1,1,0,1], fontsize=24)
mp.drawmeridians(np.arange(-180,180,60),labels=[0,0,0,1], fontsize=24)

plt.tick_params(axis='both', which='major', labelsize=24)
plt.tick_params(axis='both', which='minor', labelsize=24)
plt.xlabel('Longitude', fontsize = 24, labelpad = 30)
plt.ylabel('Latitude', fontsize = 24, labelpad = 70)

cbar = plt.colorbar(pad = 0.08)
cbar.ax.tick_params(labelsize=24)

plt.show()

#plt.pcolormesh(XG, YG,HFHR,shading='flat', vmin= -2, vmax = 2,cmap = palette)
#plt.axis([-180, 180, -80, 80])
#plt.colorbar()
#plt.show()


In [ ]:
fig2 = plt.figure(2, figsize=(18,9), dpi = 600)

palette = plt.get_cmap('RdBu_r', 50)

#x=mit.rdmds('XG'); y=mit.rdmds('YG'); e=mit.rdmds('Eta',np.Inf)
mp = Basemap(projection='merc',llcrnrlat=-80,urcrnrlat=80,\
            llcrnrlon=-180,urcrnrlon=180,lat_ts = 0, resolution='c')

h = cs.pcol(XG, YG,HRWNanom[1,:,:], projection = mp, vmin= -4, vmax = 4, cmap = palette)

mp.fillcontinents(color = 'grey')
mp.drawmapboundary()
mp.drawparallels(np.arange(-80,80,20),labels=[1,1,0,1], fontsize=24)
mp.drawmeridians(np.arange(-180,180,60),labels=[0,0,0,1], fontsize=24)

plt.tick_params(axis='both', which='major', labelsize=24)
plt.tick_params(axis='both', which='minor', labelsize=24)
plt.xlabel('Longitude', fontsize = 24, labelpad = 30)
plt.ylabel('Latitude', fontsize = 24, labelpad = 70)

cbar = plt.colorbar(pad = 0.08)
cbar.ax.tick_params(labelsize=24)

plt.show()

#plt.pcolormesh(XG, YG,HFHR,shading='flat', vmin= -2, vmax = 2,cmap = palette)
#plt.axis([-180, 180, -80, 80])
#plt.colorbar()
#plt.show()